# create producer.py

In [4]:
import BDBAProducer
import SodaConnector
import random
import time
import json

soda = SodaConnector("data.cityofnewyork.us")
requests = soda.get_data("fhrw-4uyv", limit=1000)
producer = BDBAProducer()
topic = 'ServiceRequests'

for request in requests:
    json_string = json.dumps(request)
    json_byte = b"" + json_string
    producer.send_service_request(topic, json_byte)
    time.sleep(random.randint(0, 100) * 0.1)

ModuleNotFoundError: No module named 'BDBAProducer'

# create consumer.py

In [ ]:
from kafka import KafkaConsumer
from DBHelper import DBHelper
import json
from datetime import datetime


requests = KafkaConsumer("ServiceRequests", bootstrap_servers='localhost:9092', auto_offset_reset='earliest')
db_helper = DBHelper('postgres')

for message in requests:
    message_json = json.loads(message.value)
    print(json.dumps(message_json))

    time = datetime.now()
    print(time)
    if 'agency_name' in message_json:
        agency_name =  message_json['agency_name']
        print(agency_name)
    if 'complaint_type' in message_json:
        type = message_json['complaint_type']
        print(type)
    if 'descriptor' in message_json:
        descriptor = message_json['descriptor']
        print(descriptor)
    if 'longitude' in message_json:
        longitude = message_json['longitude']
        print(longitude)
    if 'latitude' in message_json:
        latitude = message_json['latitude']
        print(latitude)

    entry = {
        'created_date': time,
        'agency_name': agency_name,
        'complaint_type': type,
        'descriptor': descriptor,
        'longitude': longitude,
        'latitude': latitude
    }
    db_helper.insert(entry, 'service_request')